# PyTorchZeroToAll course

## 1. Introduction

In [4]:
import torch
import numpy as np
print(torch.__version__)

1.5.0


## 2. Backpropogaton

In [23]:
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

w = torch.tensor([3.0], requires_grad=True)

def forward(x):
    return x * w

def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)

In [24]:
for it in range(10):
    for x_val, y_val in zip(x_data, y_data):
        L = loss(x_val, y_val)
        L.backward()
        w.data = w.data - 0.01 * w.grad.data
        # manually setting gradient data to 0, because pytorch adds the new grad data to already existing one
        w.grad.data.zero_()
        
print(w)

tensor([2.0488], requires_grad=True)


## 3. Linear regression

In [137]:
x_data = torch.tensor([1.0, 2.0, 3.0, 4.0]).resize_((4,1))
y_data = torch.tensor([2.0, 4.0, 6.0, 8.0]).resize_((4,1))

In [138]:
class MyModel(torch.nn.Module):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear = torch.nn.Linear(1,1)
        
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [154]:
model = MyModel()

In [155]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [158]:
for epoch in range(100):
    y_pred = model(x_data)
    L = loss(y_pred, y_data)
    optimizer.zero_grad()
    L.backward()
    optimizer.step()


In [159]:
model.forward(torch.tensor([[5.0]]))

tensor([[10.0000]], grad_fn=<AddmmBackward>)

## 4. Deep networks

In [174]:
from sklearn.datasets import load_digits
from sklearn import preprocessing

In [165]:
raw_data = load_digits()
raw_data.keys()

dict_keys(['DESCR', 'images', 'data', 'target_names', 'target'])

In [191]:
x_data = torch.tensor(raw_data['data']).float()
lb = preprocessing.LabelBinarizer()
lb.fit(raw_data['target'])
y_data = torch.tensor(lb.transform(raw_data['target'])).float()
print(x_data.size(), y_data.size())

torch.Size([1797, 64]) torch.Size([1797, 10])


In [236]:
class DigitsNet(torch.nn.Module):
    
    def __init__(self):
        super(DigitsNet, self).__init__()
        self.linear1 = torch.nn.Linear(64,20)
        self.linear2 = torch.nn.Linear(20,14)
        self.linear3 = torch.nn.Linear(14,10)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        out1 = self.sigmoid(self.linear1(x))
        out2 = self.sigmoid(self.linear2(out1))
        y_pred = self.sigmoid(self.linear3(out2))
        return y_pred

In [237]:
net = DigitsNet()
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

In [238]:
for i in range(10000):
    L = loss(net.forward(x_data), y_data)
    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    if (i+1) % 500 == 0:
        print("Iteration {0}: loss = {1}".format(i+1, L.item()))

Iteration 500: loss = 0.08895809203386307
Iteration 1000: loss = 0.08617626875638962
Iteration 1500: loss = 0.07686088234186172
Iteration 2000: loss = 0.06309647858142853
Iteration 2500: loss = 0.049990005791187286
Iteration 3000: loss = 0.038157302886247635
Iteration 3500: loss = 0.027999071404337883
Iteration 4000: loss = 0.02158457599580288
Iteration 4500: loss = 0.017362335696816444
Iteration 5000: loss = 0.013987812213599682
Iteration 5500: loss = 0.01091040764003992
Iteration 6000: loss = 0.008506925776600838
Iteration 6500: loss = 0.006867634132504463
Iteration 7000: loss = 0.005742269102483988
Iteration 7500: loss = 0.00493005383759737
Iteration 8000: loss = 0.004306115675717592
Iteration 8500: loss = 0.003816240234300494
Iteration 9000: loss = 0.003426517825573683
Iteration 9500: loss = 0.0031064224895089865
Iteration 10000: loss = 0.0028391522355377674


In [239]:
res = net.forward(x_data).round().detach().numpy()
true_res = np.array(y_data)
print(np.sum(np.abs(res - true_res)))

15.0
